### [예제 11] DataGeneration class 를 이용한 Diabetes 구현

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

np.random.seed(0)

In [2]:
# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

#### Diabetes class 구현

In [3]:
# Diabetes Class

class Diabetes:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, name, input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_rate):
        
        self.name = name
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden1_nodes)  
        self.b2 = np.random.rand(hidden1_nodes)
        
        # 3층 hidden layer unit  
        self.W3 = np.random.rand(hidden1_nodes, hidden2_nodes)
        self.b3 = np.random.rand(hidden2_nodes)
        
        # 4층 output layer unit  
        self.W4 = np.random.rand(hidden2_nodes, output_nodes)
        self.b4 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print(self.name, " is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(input_data)
        
        return accuracy_val
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

#### DataGeneration class 구현

In [4]:
class DataGeneration:
    
    def __init__(self, name, file_path, seperation_rate):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
    # numpy.random.shuffle()  이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기, 파일이 없는 경우 exception 발생

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))

        print("[DataGeneration]  loaded_data.shape = ", loaded_data.shape)
        
        
        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        # numpy.random.shuffle 을 이용하여 랜덤하게 데이터 섞기
        np.random.shuffle(loaded_data)
        
        # test_data 는 0 : test_data_num
        
        test_data = loaded_data[ 0:test_data_num ]

        # training_data 는 test_data_num 부터 끝까지 
        training_data = loaded_data[ test_data_num: ]

        
        return training_data, test_data

#### DataGeneration 객체를 통한 training_data , test_data 생성

In [6]:
# DataGeneration 객체 생성.
seperation_rate = 0.3 # 분리비율

data_obj = DataGeneration('Diabetes', './Diabetes.csv', seperation_rate)

(training_data, test_data) = data_obj.generate()

print('training_data.shape = ', training_data.shape, ', test_data.shape = ', test_data.shape)

[DataGeneration]  loaded_data.shape =  (759, 9)
training_data.shape =  (532, 9) , test_data.shape =  (227, 9)


#### Diabetes 객체 생성 및 학습

In [7]:
#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 3  # hidden 1 nodes 개수. 
h2_nodes = 2  # hidden 2 nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-1      # learning rate. hi_node = 2, 1e-1 에서 수렴
epochs = 20   # 반복횟수. 

# Diabetes 객체 생성
obj = Diabetes("Diabetes", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    cur_loss_val = obj.loss_val()
    print("epochs = ", step, "loss value = ", cur_loss_val)

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Diabetes  is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.2820738786178385
epochs =  1 loss value =  0.280851848789393
epochs =  2 loss value =  0.28024233793998576
epochs =  3 loss value =  0.27632800566324717
epochs =  4 loss value =  0.2563573332705407
epochs =  5 loss value =  0.19703881723939476
epochs =  6 loss value =  0.11683059401669492
epochs =  7 loss value =  0.06472601876561508
epochs =  8 loss value =  0.04037799099752729
epochs =  9 loss value =  0.028162705402986498
epochs =  10 loss value =  0.02109153777589177
epochs =  11 loss value =  0.01687486810248968
epochs =  12 loss value =  0.01427640074446576
epochs =  13 loss value =  0.012507034710050675
epochs =  14 loss value =  0.011165404744335595
epochs =  15 loss value =  0.010198203601653359
epochs =  16 loss value =  0.009883669955163262
epochs =  17 loss value =  0.010169717511987949
epochs =  18 loss value =  0.00975724106163121
epochs =  19 loss value =  0.0083992

#### test_data를 이용하여 정확도 검증

In [8]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

accuracy_ret = obj.accuracy(test_input_data, test_target_data) 

print('Accuracy => ', accuracy_ret)

Accuracy =>  0.73568281938326
